In [ ]:
!nvidia-smi

Fri Jun 21 18:07:47 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              47W / 400W |      5MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
# INSTALL ALL REQUIRED PACKAGES.

!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# IMPORT ALL THE REQUIRED LIBS.

import os
import torch
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import bitsandbytes as bnb

## **Model Fine-Tuning & Saving**

In this project, we are going to Fine-tune a custom Llama-2 model ["NousResearch/llama-2-7b-chat-hf"](https://huggingface.co/NousResearch/Llama-2-7b-chat-hf)  from Huggingface.

The dataset used is also from hugging face - ["mlabonne/guanaco-llama2-1k"](https://huggingface.co/datasets/mlabonne/guanaco-llama2-1k).

This dataset has 1000 samples and we will fine-tune the model to create a new model - "Llama-2-7b-chat".

We are going to use **LoRa** here as a fine-tuning strategy.

In [ ]:
# INIT MODEL & DATASET VARS.

model_checkpoint_hf = "NousResearch/llama-2-7b-chat-hf"

dataset_hf = "mlabonne/guanaco-llama2-1k"

new_model_checkpoint = "Llama-2-7b-chat"

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Get the dataset from HuggingFace.

dataset = datasets.load_dataset(
    dataset_hf,
    split="train"
)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
# Use BitsAndBytesConfig class to create the config for 4-bit fine-tuning.

compute_dtype = getattr(torch, "float16")
use_4bit = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,                                  # Activate 4bit precision.
    bnb_4bit_quant_type="nf4",                          # Quantization type (fp4 or nf4)
    bnb_4bit_compute_dtype=compute_dtype,               # Compute dtype for 4-bit base model.
    bnb_4bit_use_double_quant=False,                    # Activate nested quantization.
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

bnb_config

Your GPU supports bfloat16: accelerate training with bf16=True


BitsAndBytesConfig {
  "bnb_4bit_compute_dtype": "float16",
  "bnb_4bit_quant_type": "nf4",
  "load_in_4bit": true
}

In [ ]:
# Load base model

model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint_hf,
    quantization_config=bnb_config,
    device_map=device_map
)

model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LLaMA tokenizer.

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_hf, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

LlamaTokenizerFast(name_or_path='NousResearch/llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

In [ ]:
# Create LoRA config.

# Init LoRa params.
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# Get LoRa config.
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="Causal_LM"
)

peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='Causal_LM', inference_mode=False, r=64, target_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)

In [ ]:
# Set Training arguments.

# Output folder.
output_dir = "./results"

# Number of epochs or training iterations.
num_epochs = 1

# Optimizer.
optim = "paged_adamw_32bit"

# Initial learning rate (AdamW optimizer).
learning_rate = 1e-5

# Define learning rate scheduler.
lr_scheduler_type = "cosine"


train_args = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_epochs,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    optim = optim,
    weight_decay = 0.001,                    # Weight Decay to apply to all layers except all 'bias' and 'LayerNorm' weights in AdamW optimizer.
    learning_rate = learning_rate,
    save_steps = 0,
    logging_steps = 100,
    fp16 = False,                            # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    bf16 = True,                            # Whether to use bf16 16-bit (mixed) precision training instead of 32-bit training (requires Ampere or higher NVIDIA architecture).
    max_grad_norm = 0.3,                     # Gradient Clipping.
    warmup_ratio = 0.03,                     # Ratio of total training steps used for a linear warmup from 0 to learning_rate.
    group_by_length = True,
    lr_scheduler_type = lr_scheduler_type,
    report_to="tensorboard",
    remove_unused_columns=False,
)

train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=True,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strat

In [ ]:
# Use SFTTrainer (supervised fine-tuning) provided by trl from HuggingFace to start the training.

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    dataset_text_field = "text",
    max_seq_length = None,                  # Maximum sequence length
    tokenizer = tokenizer,
    args = train_args,
    packing = False,                        # Pack multiple short examples in same input sequence.
)

# Check if the dataset is empty
if len(trainer.train_dataset) == 0:
    print("Warning: The training dataset is empty. Please check your data loading process.")

In [ ]:
trainer.train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

In [ ]:
# Train model

trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,2.068600
200,1.819300


TrainOutput(global_step=250, training_loss=1.92988427734375, metrics={'train_runtime': 240.4835, 'train_samples_per_second': 4.158, 'train_steps_per_second': 1.04, 'total_flos': 8755214190673920.0, 'train_loss': 1.92988427734375, 'epoch': 1.0})

In [ ]:
# Save trained model.

trainer.model.save_pretrained(new_model_checkpoint)

In [ ]:
new_model_checkpoint

'Llama-2-7b-chat'

In [ ]:
# Save in gdrive.

!zip -r Llama-2-7b-chat.zip results Llama-2-7b-chat
!mv Llama-2-7b-chat.zip /content/drive/MyDrive/NLP-LLM/All-About-LLM/3-Fine-Tuning-Llama2-with-LoRa/training/

## **Merge Fine-tuned LoRA weights with Base Model**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_hf, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaTokenizerFast(name_or_path='NousResearch/llama-2-7b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

In [ ]:
# Load base model

base_model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint_hf,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

model = PeftModel.from_pretrained(base_model, new_model_checkpoint)
model = model.merge_and_unload()
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

## **Inference**

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256
)

In [ ]:
prompt = "What is a large language model?"

pipe(prompt)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[{'generated_text': 'What is a large language model?\nA large language model is a type of artificial intelligence (AI) model that is trained on a large corpus of text data to generate language outputs that are coherent and natural-sounding. These models are designed to learn the patterns and structures of language, and can be used for a variety of applications such as language translation, text summarization, and chatbots.\n\nLarge language models are typically trained on vast amounts of text data, such as books, articles, and websites, and are designed to learn the patterns and structures of language at a large scale. This allows them to generate text that is more coherent and natural-sounding than smaller models, which are typically trained on smaller datasets.\n\nSome examples of large language models include:\n\n1. BERT (Bidirectional Encoder Representations from Transformers): Developed by Google in 2018, BERT is a large language model that has been pre-trained on a large corpus o

In [ ]:
prompt_template = """### Instruction:
Given the following sentence, your task is to generate the negation for it in JSON format. Return only the JSON data.

### Input:
{}

### Response:
"""

sentence = "The weather forecast predicts a sunny day with a high temperature around 30 degrees Celsius, perfect for a day at the beach with friends and family."

input_sentence = prompt_template.format(sentence.strip())

result = pipe(input_sentence)

print(result[0]["generated_text"])

### Instruction:
Given the following sentence, your task is to generate the negation for it in JSON format. Return only the JSON data.

### Input:
The weather forecast predicts a sunny day with a high temperature around 30 degrees Celsius, perfect for a day at the beach with friends and family.

### Response:
{
"negation": {
"weather": {
"forecast": {
"sunny": false,
"highTemperature": 20,
"perfectForBeach": false
}
}
}
}

### Explanation:
The negation of the input sentence is created by setting the corresponding properties in the JSON object to false.

Note: The input sentence is in the format of a weather forecast, so the properties in the JSON object are named accordingly.
